In [1]:
%load_ext autoreload
%autoreload 2
# %env CUDA_DEVICE_ORDER=PCI_BUS_ID
# %env CUDA_VISIBLE_DEVICES=1
# %env CUDA_LAUNCH_BLOCKING=1
%env MKL_NUM_THREADS=1
%env OMP_NUM_THREADS=1
%env NUMEXPR_NUM_THREADS=1
%env MKL_DEBUG_CPU_TYPE=5

env: MKL_NUM_THREADS=1
env: OMP_NUM_THREADS=1
env: NUMEXPR_NUM_THREADS=1
env: MKL_DEBUG_CPU_TYPE=5


# Set up the gym and wrap a monitor around it that will periodically record movies as it learns

In [2]:
import gym
from surface_seg.envs.mcs_env import MCSEnv
from surface_seg.utils.callback_simple import Callback
from tensorforce.execution import Runner
import gym.wrappers
import numpy as np
import tensorforce
import copy
import tensorflow as tf
import os

In [3]:
timesteps = 500
save_dir = './result_multi_env/test/'
multi_env = True

In [4]:
def setup_env(recording=True):
    
    # Set up gym
    MCS_gym = MCSEnv(observation_fingerprints=False, 
                     observation_forces=False,
                     permute_seed=42,
                     save_dir = save_dir,
                     timesteps = timesteps,
                     save_every = 2,
                     plot_every = 2,
                     multi_env = multi_env
                    )
    
    if recording:
    # Wrap the gym to provide video rendering every 50 steps
        MCS_gym = gym.wrappers.Monitor(MCS_gym, 
                                         os.path.join(save_dir, 'vid'), 
                                         force=True,
                                        video_callable = lambda episode_id: (episode_id+1)%30==0) #every 50, starting at 51
    
    #Convert gym to tensorforce environment
    env = tensorforce.environments.OpenAIGym(MCS_gym,
                                         max_episode_timesteps=timesteps,
                                         visualize=False)
    
    return env
    
env = setup_env().environment.env
print('initial energy', env.initial_energy)
print('thermal energy', env.thermal_energy)
n = 3
print('%dKT' %n, n * env.thermal_energy)

initial energy 4.100605630583811
thermal energy 0.827232
3KT 2.481696


/home/junwoony/miniconda3/envs/arpae/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Trying to monitor an environment which has no 'spec' set. This usually means you did not create it via 'gym.make', and is recommended only for advanced users.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


# Set up the gym and agent in tensorforce

In [5]:
from tensorforce.agents import Agent

agent = Agent.create(
    agent='trpo', 
    environment=setup_env(), 
    batch_size=1, 
    learning_rate=1e-3,
    memory = 40000,
    max_episode_timesteps = timesteps,
    exploration=dict(
        type='decaying', unit='timesteps', decay='exponential',
        initial_value=0.2, decay_steps=1000000, decay_rate=0.5
    ),
    recorder = dict(
        directory = os.path.join(save_dir, 'recorder'), frequency=1), #required for recording states and actions
    summarizer = dict(
        directory = os.path.join(save_dir, 'tb'), labels='all', frequency=1, #Tensorboard summarizer
    )
)
    

agent_spec = agent.spec

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.while_loop(c, b, vars, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.while_loop(c, b, vars))
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


# Run the DRL method in parallel (multiple environments)

In [ ]:
callback = Callback(save_dir).episode_finish

num_parallel = 32
runner = Runner(
    agent=agent_spec,
    environments=[setup_env(recording=True) for _ in range(num_parallel)],
    num_parallel=num_parallel,
    remote='multiprocessing',
    max_episode_timesteps=timesteps,
)

runner.run(num_episodes=num_parallel*500, callback=callback, callback_episode_frequency=20)
# runner.run(num_episodes=100, evaluation=True)
runner.close()

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


Episodes:   0%|          | 0/16000 [00:00, reward=0.00, ts/ep=0, sec/ep=0.00, ms/ts=0.0, agent=0.0%, comm=0.0%]Exception in thread Thread-39:
Traceback (most recent call last):
  File "/home/junwoony/miniconda3/envs/arpae/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/home/junwoony/miniconda3/envs/arpae/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/junwoony/miniconda3/envs/arpae/lib/python3.7/site-packages/tensorforce/environments/environment.py", line 570, in finish_execute
    self.observation = self.execute(actions=actions)
  File "/home/junwoony/miniconda3/envs/arpae/lib/python3.7/site-packages/tensorforce/environments/environment.py", line 540, in execute
    states, terminal, reward, seconds = self.receive(function='execute')
  File "/home/junwoony/miniconda3/envs/arpae/lib/python3.7/site-packages/tensorforce/environments/environment.py", line 501, in receive
    raise TensorforceError

Exception in thread Thread-90:
Traceback (most recent call last):
  File "/home/junwoony/miniconda3/envs/arpae/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/home/junwoony/miniconda3/envs/arpae/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/junwoony/miniconda3/envs/arpae/lib/python3.7/site-packages/tensorforce/environments/environment.py", line 570, in finish_execute
    self.observation = self.execute(actions=actions)
  File "/home/junwoony/miniconda3/envs/arpae/lib/python3.7/site-packages/tensorforce/environments/environment.py", line 540, in execute
    states, terminal, reward, seconds = self.receive(function='execute')
  File "/home/junwoony/miniconda3/envs/arpae/lib/python3.7/site-packages/tensorforce/environments/environment.py", line 501, in receive
    raise TensorforceError(message='{}: {}'.format(etype, value)).with_traceback(traceback)
TypeError: __traceback__ must be a traceback 